---
# Projectr : 하루시작 지하철 혼잡도 분석
### Description : 
- <span style = "color : #FFBE98">서울교통공사 1, 2호선 역별 일별 시간대별 승하차인원 정보 </span>
    
### Author : Zenzen
### Date : 2024. 06. 03. (Mon) ~
### Detail : 
### Update: 
- 2024.06.03 zenzen : 일별 데이터 정제 <br><br>
  - 기존 테스트 데이터 의 일별 데이터가 없는 부분 참고하여 일별 데이터 추출 및 데이터 칼럼 정제를 시작.


---

In [1]:
import warnings

warnings.filterwarnings('ignore')

# Data 정제

## Kakao Open API 이용하여 위・경도 반환하기

In [2]:
import pandas as pd

seoul_subway = pd.read_csv("../Data/StationInfo_2022-12-31.csv")

seoul_subway.head()

,Unnamed: 0,역명,고유역번호(외부역코드),승하차구분,역_이름코드
0,0,서울역,150,승차,서울역_150
1,1,시청,151,승차,시청_151
2,2,종각,152,승차,종각_152
3,3,종로3가,153,승차,종로3가_153
4,4,종로5가,154,승차,종로5가_154


### '역명' Column Data 중에서 끝자리가 '역' 단어로 끝나는 역명 검색하기.

In [3]:
# import re

# pattern = r'\w*역$'  # 임의의 문자(단어 문자)가 0번 이상 반복되고 '역'으로 끝나는 패턴

# matches = seoul_subway['역명'].str.findall(pattern)

# seoul_subway[seoul_subway['역명'].str.contains(pattern)]

In [4]:
import requests
import re

# 괄호 안 내용 제거
seoul_subway['역명'] = seoul_subway['역명'].apply(lambda x: re.sub(r'\([^)]*\)', '', x) if '(' in x else x)

# '역명' Column Data 중에서 끝자리에 '역' 단어가 없는 경우에만 '역'을 추가
seoul_subway['역명'] = seoul_subway['역명'].apply(lambda x: x + '역' if not x.endswith('역') else x)

x = []  # List of latitude
y = []  # List of longitude

for z in seoul_subway['역명']:
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(z)
    headers = {
        "Authorization": "KakaoAK cdd06f3d902860e8428121d4b65b7d04"
    }

    # JSON 형태로 데이터가 주어진다
    places = requests.get(url, headers=headers).json()['documents'][0]
    x.append(places['y'])
    y.append(places['x'])

seoul_subway['latitude'] = x
seoul_subway['longitude'] = y
seoul_subway_latlon = seoul_subway.reset_index()

seoul_subway_latlon.head()

,index,Unnamed: 0,역명,고유역번호(외부역코드),승하차구분,역_이름코드,latitude,longitude
0,0,0,서울역,150,승차,서울역_150,37.5546788388674,126.970606917394
1,1,1,시청역,151,승차,시청_151,37.56534539636417,126.97719821079865
2,2,2,종각역,152,승차,종각_152,37.570227990912244,126.98315081716676
3,3,3,종로3가역,153,승차,종로3가_153,37.570420844523,126.992153252476
4,4,4,종로5가역,154,승차,종로5가_154,37.57097610838373,127.00153834521934


In [7]:
seoul_subway_latlon_2 = seoul_subway_latlon.iloc[:, 2:]

In [9]:
seoul_subway_latlon_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272 entries, 0 to 271
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   역명            272 non-null    object
 1   고유역번호(외부역코드)  272 non-null    int64 
 2   승하차구분         272 non-null    object
 3   역_이름코드        272 non-null    object
 4   latitude      272 non-null    object
 5   longitude     272 non-null    object
dtypes: int64(1), object(5)
memory usage: 12.9+ KB


In [12]:
seoul_subway_latlon_2['latitude'] = seoul_subway_latlon_2['latitude'].astype(float)
seoul_subway_latlon_2['longitude'] = seoul_subway_latlon_2['longitude'].astype(float)

In [13]:
seoul_subway_latlon_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272 entries, 0 to 271
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   역명            272 non-null    object 
 1   고유역번호(외부역코드)  272 non-null    int64  
 2   승하차구분         272 non-null    object 
 3   역_이름코드        272 non-null    object 
 4   latitude      272 non-null    float64
 5   longitude     272 non-null    float64
dtypes: float64(2), int64(1), object(3)
memory usage: 12.9+ KB


In [18]:
seoul_subway_latlon_2.to_csv("../Data/seoul_subway_latlon_zenzen.csv", index = False, encoding = 'utf-8')

In [19]:
pd.read_csv("../Data/seoul_subway_latlon_zenzen.csv")

,역명,고유역번호(외부역코드),승하차구분,역_이름코드,latitude,longitude
0,서울역,150,승차,서울역_150,37.554679,126.970607
1,시청역,151,승차,시청_151,37.565345,126.977198
2,종각역,152,승차,종각_152,37.570228,126.983151
3,종로3가역,153,승차,종로3가_153,37.570421,126.992153
4,종로5가역,154,승차,종로5가_154,37.570976,127.001538
...,...,...,...,...,...,...
267,단대오거리역,2824,승차,단대오거리_2824,37.445022,127.156770
268,신흥역,2825,승차,신흥_2825,37.440923,127.147538
269,수진역,2826,승차,수진_2826,37.437378,127.140535
270,모란역,2827,승차,모란_2827,37.434038,127.129970
